In [134]:
class Player: 
    def __init__(self, name, initial_wealth):
        self.name = name
        self.wealth = initial_wealth
        
        # Initializing the car parts, each key is mapped to a quality fron 0 to 10, which is empty when initialized (before trading)
        self.car_parts = {"Engine": None,
            "Wheels": None,   
            "Suspension" : None,
            "Brakes" : None,
            "Chassis" : None,
            "Cooling System": None,
            "Transmission": None}
        self.points = 0
        self.pilot = None

    
    def buy_car_part(self, car_part, quality, price):
        if self.wealth >= price:
            self.car_parts[car_part] = quality
            self.wealth -= price
            print("Bought", car_part, "with quality", quality, "for $", price)
        else:
            print("Insufficient funds to buy the car part.")

            
    def sell_car_part(self, car_part, price):
        if car_part in self.car_parts:
            self.car_parts[car_part] = None
            self.wealth += price
            print("Sold", car_part, "for $", price)
        else:
            print("You don't have", car_part, "to sell.")
            

    def update_points(self, race_points):
        self.points += race_points
        
    
    def sign_pilot(self, pilots, team):
        print("Sign a pilot among the following list, their price is equal to their skill.")
        for i in range(len(pilots)):
            print(str(i+1)+ " Name:", pilots[i].name + ", Skill: " + str(pilots[i].skill) + ", Confidence: " + str(pilots[i].confidence))
        
        # I think there are much more elegant ways to do the part below (keep asking for valid entry) but this is the most intuitive one I could think of.
        booly = True
        while booly:
            try:
                choice = int(input("Choose a pilot"))
                booly = False
            except:
                print("Invalid choice. Please select a pilot from 1 to 10.")
                booly = True
        self.pilot = pilots[choice-1]
        self.wealth -= pilots[choice-1].skill
        pilots[choice-1].team=team
        pilots.pop(choice-1)

    def display_status(self):
        print(self.name, "currently has $", self.wealth)
        print("Car Parts:")
        for car_part, quality in self.car_parts.items():
            print("-", car_part+ ": Quality", quality)
        print("Points:", self.points)
        
class Pilot:
    def __init__(self, name, confidence, skill, team):
        self.name = name
        self.confidence = confidence
        self.skill = skill 
        self.team = team
    
    def display_status(self):
        print("Name:", self.name + ", Skill:" + str(self.skill)+ ", Confidence: " + str(self.confidence)+ ", Team: " + str(self.team))

pilots = [
    Pilot("Lewis Hamilton", 80, 95, None),
    Pilot("Max Verstappen", 92, 92, None),
    Pilot("Fernando Alonso", 89, 92, None),
    Pilot("Charles Leclerc", 81, 90, None),
    Pilot("Carlos Sainz", 80, 88, None),
    Pilot("Lando Norris", 79, 87, None),
    Pilot("Sergio Perez", 66, 82, None),
    Pilot("George Russell", 82, 86, None),
    Pilot("Kevin Magnussen", 85, 80, None),
    Pilot("Daniel Ricciardo", 70, 80, None)
]

# Example usage:
#player = Player(input("Enter your name"),1000)
#player.sign_pilot(pilots)
#player.pilot.display_status()
#player.buy_car_part("Engine", 8, 200)
#player.buy_car_part("Tires", 7, 150)
#player.display_status()
#player.sell_car_part("Engine", 250)
#player.display_status()



In [135]:
import random

class Race:
    def __init__(self, name, location, weather, rain, path):
        self.name = name
        self.location = location
        self.weather = weather
        self.rain = rain
        self.results = []
        self.path = path

    def start_race(self, players):
        print(f"Starting the race: {self.name} at {self.location} with {self.weather} weather.")
        
        # Simulate the race by calculating each player's performance score
        
        for player in players:
            car_quality = sum(player.car_parts.values())
            print(car_quality , player.pilot.skill , player.pilot.confidence)
            performance_score = car_quality + player.pilot.skill + player.pilot.confidence + self.rain*player.car_parts.get("Wheels", 0) + self.weather*player.car_parts.get("Cooling System", 0)/10 + random.uniform(-5, 5)
            self.results.append((player, performance_score))

        # Sort results by performance_score in descending order (higher is better)
        self.results.sort(key=lambda x: x[1], reverse=True)

        #Determine race outcome and update player points
        points_distribution = [25, 18, 15, 12, 10, 8, 6, 4, 2, 1]  
        for i, (player, score) in enumerate(self.results):
            if i < len(points_distribution):
                player.update_points(points_distribution[i])
                print(f"{player.name} finished in position {i+1} and earned {points_distribution[i]} points.")
            else:
                print(f"{player.name} finished in position {i+1} and earned 0 points.")

        print("Race completed!\n")

    def generate_race_commentary(self):
        with open(self.path, 'r') as file:
            lines = file.readlines()
        x= [line.strip() for line in lines]
        ranking=self.results.copy()
        winner=ranking[0][0].pilot.name
        for i in x:
            modified_string=i
            while "Pilot XYZ" in modified_string:
                name=ranking[0][0].pilot.name
                start_index = modified_string.find("Pilot XYZ")
                end_index = start_index + len("Pilot XYZ")
                modified_string = modified_string[:start_index] + name + modified_string[end_index:]
                ranking.pop(0)
            if "Pilot WXYZ" in i:
                start_index = modified_string.find("Pilot WXYZ")
                end_index = start_index + len("Pilot WXYZ")
                modified_string = modified_string[:start_index] + winner + modified_string[end_index:]
            print(modified_string)
        

    def update_player_status(self, players):
        
        for player in players:
            for car_part, quality in player.car_parts.items():
                player.car_parts[car_part] = max(0, quality - 1)
            player.display_status()
            
            
    def pre_race_interaction(self, players):
        print("Hello!", players[0].name , ", we are currently at the", self.name, ". What advice would like to give to your pilot?")
        print("Press A to motivate him")
        print("Press B to warn him about recent results")
        
        booly = True
        while booly:
            choice = input("Enter your choice: ")
            if choice == "A":
                values = [2, -1]
                probabilities = [0.9, 0.1]
                x = random.choices(values, probabilities)[0]
                players[0].pilot.confidence += x
                if x > 0:
                    print("Your motivating words made your pilot more confident!")
                else:
                    print("Your words made your pilot over-confident, humbleness is key!")
                booly = False

            elif choice == "B":
                values = [-2, 1]
                probabilities = [0.9, 0.1]
                x = random.choices(values, probabilities)[0]
                players[0].pilot.confidence += x
                if x < 0:
                    print("Your harsh warning made your pilot lose self-confidence!")
                else:
                    print("Your harsh warning made your pilot eager to show what he is truly capable of!")
                booly = False
            else:
                print("Invalid choice. Please try again.")
            
            

    def run_race(self, players):
        self.pre_race_interaction(players)
        self.start_race(players)
        self.generate_race_commentary()
        self.update_player_status(players)






#Monaco = Race("Monaco Grand Prix", "Europe", 20, 0)
#Monaco.run_race(players)

In [136]:
from PIL import Image, ImageDraw, ImageFont

def podiumpic(race): 
    image_path = 'podiumf1.jpg'
    image = Image.open(image_path)

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Define text to be written
    text1 = "2nd: " + race.results[0][0].name
    text2 = "1st: " + race.results[1][0].name
    text3 = "3rd: " + race.results[2][0].name
    text4 = race.name + " Podium"

    
    font_path = "Arial.ttf"  
    font_size = 37
    fontsize_gp = 60
    font = ImageFont.truetype(font_path, font_size)
    fontgp = ImageFont.truetype(font_path, fontsize_gp)
    text_color = (0, 0, 0)  # Black color

    text_position1 = (40, 515)  # (x, y) coordinates
    text_position2 = (857, 352)
    text_position3 = (1628, 471)
    text_position4 = (621, 41)
    draw.text(text_position1, text1, fill=text_color, font=font)
    draw.text(text_position2, text2, fill=text_color, font=font)
    draw.text(text_position3, text3, fill=text_color, font=font)
    draw.text(text_position4, text4, fill=text_color, font=fontgp)

    image.show()


In [137]:
import yfinance as yf
from datetime import datetime

class Market:
    def __init__(self):
        
        # Define symbols for different car parts
        self.symbols = {
            "Engine": [["AAPL", 6], ["MCD", 8], ["HD", 10]],   
            "Wheels": [["GOOG", 6], ["V", 8], ["ACN", 10]],     
            "Suspension": [["AMZN", 6], ["QCOM", 8], ["CAT", 10]],   
            "Brakes": [["JPM", 6], ["DHR", 8], ["AMGN", 10]],       
            "Chassis": [["TSLA", 6], ["AXP", 8], ["ETN", 10]],     
            "Cooling System": [["XOM", 6], ["UNP", 8], ["SYK", 10]],  
            "Transmission": [["ORCL", 6], ["LOW", 8], ["DE", 10]]   
        }

    def get_stock_price(self, car_part, date):
        
        symbols = self.symbols.get(car_part)
        symbolz = []
        for x in symbols:
            symbolz.append(x[0])
        if symbolz:
            # Fetch historical stock data from Yahoo Finance for each symbol
            prices = []
            for symbol in symbolz:
                stock_data = yf.Ticker(symbol)
                start_date = datetime.strptime(date, "%Y-%m-%d")
                stock_data = stock_data.history(start=start_date).iloc[0]
                if not stock_data.empty:
                    price = stock_data["Close"]
                    prices.append(price)
                else:
                    print(f"No data available for {date} for {car_part} ({symbol}).")
                    return None
            return prices
        else:
            print(f"No stock symbols found for {car_part}.")
            return None


def mainplayertrades(player, market, date):
    # Check if the player has any car parts
    while any(value is None for value in player.car_parts.values()):
        print("You don't have all car parts. You must buy car parts from the market.")

        
        for car_part, symbols in market.symbols.items():
            if symbols:  
                print(f"{car_part}:")
                count = 0
                for symbol, quality in symbols:
                    price = market.get_stock_price(car_part, date)
                    price = price[count % 3]
                    print(f"- Stock: {symbol}, Quality: {quality}, Price: {price}")
                    count += 1

        # Prompt the player to choose a car part to buy from the market
        chosen_car_part = input("Enter the name of the car part you want to buy: ")
        while chosen_car_part not in market.symbols:
            print("Invalid choice")
            chosen_car_part = input("Enter the name of the car part you want to buy: ")
        chosen_symbol = int(input("Enter the position of the item you want to buy from (1/2/3): "))
        chosen_price = market.get_stock_price(chosen_car_part, date)[chosen_symbol - 1]
        player.buy_car_part(chosen_car_part, market.symbols[chosen_car_part][chosen_symbol - 1][1], chosen_price)
        player.display_status()
        # Remove all instances of the chosen item from the list of available car parts
        market.symbols[chosen_car_part] = []

    else:
        print("You have the following car parts:")
        for car_part, quality in player.car_parts.items():
            print(f"- {car_part}: Quality {quality}")

        # Give the player the option to buy a car part or sell an existing one
        action = input("Do you want to buy or sell a car part? (buy/sell/none): ").lower()
        if action == "buy":
            # Display available car parts and their prices from the market
            for car_part, symbols in market.symbols.items():
                if symbols:  # Check if there are available stocks for this car part
                    print(f"{car_part}:")
                    for symbol, quality in symbols:
                        price = market.get_stock_price(car_part, date)
                        print(f"- Stock: {symbol}, Quality: {quality}, Price: {price}")

            # Prompt the player to choose a car part to buy from the market
            chosen_car_part = input("Enter the name of the car part you want to buy: ")
            chosen_symbol = int(input("Enter the stock symbol you want to buy from: "))
            chosen_price = market.get_stock_price(chosen_car_part, date)[chosen_symbol - 1]
            if not (player.car_parts[chosen_car_part] is None):
                current_quality = player.car_parts[chosen_car_part]
                print(current_quality)
                prices = market.get_stock_price(chosen_car_part, date) 
                selling_price = (min(prices)) * current_quality /10
                player.sell_car_part(chosen_car_part, selling_price)
            player.buy_car_part(chosen_car_part, market.symbols[chosen_car_part][chosen_symbol - 1][1], chosen_price)
            
            player.display_status()

        elif action == "sell":
            # Prompt the player to choose a car part to sell
            chosen_car_part = input("Enter the name of the car part you want to sell: ")
            if chosen_car_part in player.car_parts:
                current_quality = player.car_parts[chosen_car_part]
                print(current_quality)
                prices = market.get_stock_price(chosen_car_part, date) 
                selling_price = (min(prices)) * current_quality /10
                player.sell_car_part(chosen_car_part, selling_price)
            else:
                print("You don't have the specified car part to sell.")
        else:
            print("No action chosen. Continuing without trading.")




In [ ]:
number_of_players = int(input("Welcome to the Free Trading Formula 1 Competition! How many players are with us today?"))
market = Market()
date = "2024-01-01"
players = []

for i in range(number_of_players):
    player = Player(input("Enter your name"),10000)
    player.sign_pilot(pilots, i+1)
    player.pilot.display_status()
    while any(value is None for value in player.car_parts.values()) or continuer_trade:
        mainplayertrades(player, market, date)
        continuer_trade = int(input("Do you want to continue trading? (0 for No, 1 for Yes)"))
    player.display_status()
    players.append(player)




Welcome to the Free Trading Formula 1 Competition! How many players are with us today? 1
Enter your name chady


Sign a pilot among the following list, their price is equal to their skill.
1 Name: Lewis Hamilton, Skill: 95, Confidence: 80
2 Name: Max Verstappen, Skill: 92, Confidence: 92
3 Name: Fernando Alonso, Skill: 92, Confidence: 89
4 Name: Charles Leclerc, Skill: 90, Confidence: 81
5 Name: Carlos Sainz, Skill: 88, Confidence: 80
6 Name: Lando Norris, Skill: 87, Confidence: 79
7 Name: Sergio Perez, Skill: 82, Confidence: 66
8 Name: George Russell, Skill: 86, Confidence: 82
9 Name: Kevin Magnussen, Skill: 80, Confidence: 85
10 Name: Daniel Ricciardo, Skill: 80, Confidence: 70


Choose a pilot 1


Name: Lewis Hamilton, Skill:95, Confidence: 80, Team: 1
You don't have all car parts. You must buy car parts from the market.
Engine:
- Stock: AAPL, Quality: 6, Price: 185.15228271484375
- Stock: MCD, Quality: 8, Price: 295.3595275878906
- Stock: HD, Quality: 10, Price: 343.02838134765625
Wheels:
- Stock: GOOG, Quality: 6, Price: 139.55999755859375
- Stock: V, Quality: 8, Price: 257.9108581542969
- Stock: ACN, Quality: 10, Price: 344.3245544433594
Suspension:
- Stock: AMZN, Quality: 6, Price: 149.92999267578125
- Stock: QCOM, Quality: 8, Price: 139.5213165283203
- Stock: CAT, Quality: 10, Price: 290.3019104003906
Brakes:
- Stock: JPM, Quality: 6, Price: 170.03358459472656
- Stock: DHR, Quality: 8, Price: 234.53485107421875
- Stock: AMGN, Quality: 10, Price: 293.0021667480469
Chassis:
- Stock: TSLA, Quality: 6, Price: 248.4199981689453
- Stock: AXP, Quality: 8, Price: 187.1238250732422
- Stock: ETN, Quality: 10, Price: 237.44418334960938
Cooling System:
- Stock: XOM, Quality: 6, Price: 

Enter the name of the car part you want to buy:  Chassis
Enter the position of the item you want to buy from (1/2/3):  2


Bought Chassis with quality 8 for $ 187.1238250732422
chady currently has $ 9717.876174926758
Car Parts:
- Engine: Quality None
- Wheels: Quality None
- Suspension: Quality None
- Brakes: Quality None
- Chassis: Quality 8
- Cooling System: Quality None
- Transmission: Quality None
Points: 0
You don't have all car parts. You must buy car parts from the market.
Engine:
- Stock: AAPL, Quality: 6, Price: 185.15228271484375
- Stock: MCD, Quality: 8, Price: 295.3595275878906
- Stock: HD, Quality: 10, Price: 343.02838134765625
Wheels:
- Stock: GOOG, Quality: 6, Price: 139.55999755859375
- Stock: V, Quality: 8, Price: 257.9108581542969
- Stock: ACN, Quality: 10, Price: 344.3245849609375
Suspension:
- Stock: AMZN, Quality: 6, Price: 149.92999267578125
- Stock: QCOM, Quality: 8, Price: 139.5213165283203
- Stock: CAT, Quality: 10, Price: 290.3019104003906
Brakes:
- Stock: JPM, Quality: 6, Price: 170.0335693359375
- Stock: DHR, Quality: 8, Price: 234.53485107421875
- Stock: AMGN, Quality: 10, Pric

In [107]:
market = Market()

for i in range(10-number_of_players):
    playeri = Player("John " + str(i+number_of_players), 10000)
    playeri.pilot=pilots[i-1]
    pilots[i-1].team=i+number_of_players+1
    playeri.pilot.display_status()
    for keyy in playeri.car_parts:
        x = random.choice([1,2,3])
        prix = market.get_stock_price(keyy, date)[ x - 1]
        playeri.buy_car_part(keyy, market.symbols[keyy][x-1][1], prix)
    playeri.display_status()
    players.append(playeri)
    


Name: Daniel Ricciardo, Skill:80, Confidence: 70, Team: 2
Bought Engine with quality 10 for $ 343.02838134765625
Bought Wheels with quality 6 for $ 139.55999755859375
Bought Suspension with quality 6 for $ 149.92999267578125
Bought Brakes with quality 8 for $ 234.53485107421875
Bought Chassis with quality 10 for $ 237.44418334960938
Bought Cooling System with quality 6 for $ 100.600341796875
Bought Transmission with quality 6 for $ 103.32489776611328
John 1 currently has $ 8691.577354431152
Car Parts:
- Engine: Quality 10
- Wheels: Quality 6
- Suspension: Quality 6
- Brakes: Quality 8
- Chassis: Quality 10
- Cooling System: Quality 6
- Transmission: Quality 6
Points: 0
Name: Max Verstappen, Skill:92, Confidence: 92, Team: 3
Bought Engine with quality 6 for $ 185.15228271484375
Bought Wheels with quality 10 for $ 344.3245544433594
Bought Suspension with quality 6 for $ 149.92999267578125
Bought Brakes with quality 6 for $ 170.0335693359375
Bought Chassis with quality 8 for $ 187.1238250

In [108]:
print(players)
Monaco = Race("Monaco Grand Prix", "Europe", 20, 0, 'race1.txt')
Monaco.run_race(players)
podiumpic(Monaco)

date=date = "2024-02-01"
Las_Vegas = Race("Las Vegas Grand Prix", "Europe", 32, 0, 'race2.txt')
mainplayertrades(player, market, date)
Las_Vegas.run_race(players)
podiumpic(Las_Vegas)

date=date = "2024-03-01"
France = Race("France Grand Prix", "Europe", 25, 0, 'race3.txt')
mainplayertrades(player, market, date)
France.run_race(players)
podiumpic(France)

date=date = "2024-04-01"
Dubai = Race("Dubai Grand Prix", "Europe", 40, 0, 'race4.txt')
mainplayertrades(player, market, date)
Dubai.run_race(players)
podiumpic(Dubai)

date=date = "2024-05-01"
Japan = Race("Japan Grand Prix", "Europe", 12, 1, 'race5.txt')
mainplayertrades(player, market, date)
Japan.run_race(players)
podiumpic(Japan)

winner = max(players, key=lambda p: p.points)
print(f"The winner of the competition is {winner.name} with {winner.points} points!")



[<__main__.Player object at 0x1620d67d0>, <__main__.Player object at 0x161d9a750>, <__main__.Player object at 0x1620dee50>, <__main__.Player object at 0x13fae6c50>, <__main__.Player object at 0x13b3ced10>, <__main__.Player object at 0x13ff4ba10>, <__main__.Player object at 0x13fad9510>, <__main__.Player object at 0x13fe5eb90>, <__main__.Player object at 0x161dbfe10>, <__main__.Player object at 0x161d87dd0>]
Hello! Chady , we are currently at the Monaco Grand Prix . What advice would like to give to your pilot?
Press A to motivate him
Press B to warn him about recent results


Enter your choice:  A


Your motivating words made your pilot more confident!
Starting the race: Monaco Grand Prix at Europe with 20 weather.
50 95 82
52 80 70
56 92 92
58 92 89
60 90 81
64 88 80
52 87 79
52 82 66
54 86 82
54 80 85
John 2 finished in position 1 and earned 25 points.
John 5 finished in position 2 and earned 18 points.
John 3 finished in position 3 and earned 15 points.
John 4 finished in position 4 and earned 12 points.
Chady finished in position 5 and earned 10 points.
John 6 finished in position 6 and earned 8 points.
John 8 finished in position 7 and earned 6 points.
John 9 finished in position 8 and earned 4 points.
John 1 finished in position 9 and earned 2 points.
John 7 finished in position 10 and earned 1 points.
Race completed!

The race begins with a burst of speed! The drivers are jostling for position as they head into the first corner.
There's a tight battle for the lead, with several pilots trading places in the opening laps.
Max Verstappen pulls ahead with a daring move, but Car

KeyboardInterrupt: Interrupted by user